In [1]:
import os

!git clone https://github.com/QingruZhang/AdaLoRA.git
!pip install ./AdaLoRA/loralib
!pip install transformers datasets evaluate accelerate scikit-learn bitsandbytes
!pip install --upgrade pyarrow 

print("Installation complete. If you get a pyarrow error below, RESTART KERNEL and run again.")

fatal: destination path 'AdaLoRA' already exists and is not an empty directory.
Processing ./AdaLoRA/loralib
  Preparing metadata (setup.py) ... done
  Created wheel for loralib: filename=loralib-0.1.0-py3-none-any.whl size=13657 sha256=c41f26e3b2548a53138644cdbdf153e48942cd940c183be9279845222af521f7
  Stored in directory: /tmp/pip-ephem-wheel-cache-bn2vszs1/wheels/a4/32/19/31e636f5205565008b63240ad971590477bdbce7da08342170
Successfully built loralib
  Attempting uninstall: loralib
    Found existing installation: loralib 0.1.0
    Uninstalling loralib-0.1.0:
      Successfully uninstalled loralib-0.1.0
  Using cached pyarrow-22.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.2 kB)
Using cached pyarrow-22.0.0-cp312-cp312-manylinux_2_28_x86_64.whl (47.7 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 15.0.2
    Uninstalling pyarrow-15.0.2:
      Successfully uninstalled pyarrow-15.0.2
Installation complete. If you get a pyarrow error below, RESTART KER

In [2]:
!pip install datasets
!pip install transformers
!pip install -U peft
!pip install evaluate
!pip install -U datasets==2.20.0 pyarrow==15.0.2 transformers==4.44.2 evaluate==0.4.2 --no-cache-dir
!pip install -q datasets evaluate accelerate scikit-learn pandas matplotlib
!pip install -U "transformers>=4.41" accelerate safetensors
!pip install -U bitsandbytes


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 253.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 243.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 249.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 22.0.0
    Uninstalling pyarrow-22.0.0:
      Successfully uninstalled pyarrow-22.0.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.3
    Uninstalling transformers-4.57.3:
      Successfully uninstalled transformers-4.57.3
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
  Using cached tokenizers-0.22.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.w

In [3]:
import warnings
import os
import logging
os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
warnings.filterwarnings("ignore")

In [4]:
import os
import warnings
import time

os.environ.setdefault("CUDA_VISIBLE_DEVICES", "0")

os.environ["WANDB_DISABLED"] = "true"
os.environ["TRANSFORMERS_NO_ADVISORY_WARNINGS"] = "true"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

warnings.filterwarnings("ignore")

import torch
import numpy as np
import evaluate
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    logging as hf_logging,
)
from peft import get_peft_model, AdaLoraConfig, TaskType

def load_dataset_with_retry(path, name=None, max_retries=None, wait=2):
    attempt = 0
    while True:
        try:
            if name is not None:
                ds = load_dataset(path, name)
            else:
                ds = load_dataset(path)
            print(f"✅ Loaded dataset '{path}'" + (f"/'{name}'" if name else "") + " successfully.")
            return ds
        except Exception as e:
            attempt += 1
            wait_time = wait * min(5, attempt)  
            print(f"[Attempt {attempt}] Failed to load dataset: {e}")
            print(f"Retrying in {wait_time} seconds...\n")
            if max_retries is not None and attempt >= max_retries:
                print("Max retries reached. Raising error.")
                raise
            time.sleep(wait_time)


def main():

    MODEL_ID = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
    TASK_NAME = "sst2"
    OUTPUT_DIR = "./SST2_adalora_results"

    NUM_EPOCHS = 3
    TRAIN_BATCH_SIZE = 16
    EVAL_BATCH_SIZE = 8
    LEARNING_RATE = 2e-4
    MAX_LEN = 256
    RANK = 8  

    hf_logging.set_verbosity_error()

    print("torch:", torch.__version__)
    print("cuda devices:", torch.cuda.device_count())

    print("📚 Loading GLUE/SST-2 dataset with retry...")
    raw_datasets = load_dataset_with_retry("glue", TASK_NAME)


    print("🧠 Loading base TinyLlama model...")
    id2label = {0: "Negative", 1: "Positive"}
    label2id = {"Negative": 0, "Positive": 1}

    device = "cuda" if torch.cuda.is_available() else "cpu"

    model = AutoModelForSequenceClassification.from_pretrained(
        MODEL_ID,
        num_labels=2,
        id2label=id2label,
        label2id=label2id,
        torch_dtype=torch.bfloat16 if torch.cuda.is_available() else torch.float32,
    )

    print("🔤 Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, use_fast=True)

    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.resize_token_embeddings(len(tokenizer))

    model.config.pad_token_id = tokenizer.pad_token_id
    model.config.use_cache = False  

    model.to(device)


    print("🧹 Preprocessing & tokenizing...")

    def clean_text(s):
        if s is None:
            return ""
        return " ".join(str(s).strip().split())

    def preprocess_function(batch):
        sentences = [clean_text(s) for s in batch["sentence"]]
        return tokenizer(
            sentences,
            truncation=True,
            max_length=MAX_LEN,
        )

    tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

    drop_cols = [c for c in ["sentence", "idx"] if c in tokenized_datasets["train"].column_names]
    if drop_cols:
        tokenized_datasets = tokenized_datasets.remove_columns(drop_cols)

    if "label" in tokenized_datasets["train"].column_names and "labels" not in tokenized_datasets["train"].column_names:
        tokenized_datasets = tokenized_datasets.rename_column("label", "labels")

    tokenized_datasets.set_format("torch")

    train_dataset = tokenized_datasets["train"]
    if len(train_dataset) > 10000:
        train_dataset = train_dataset.select(range(10000))
    eval_dataset = tokenized_datasets["validation"]


    steps_per_epoch = (len(train_dataset) + TRAIN_BATCH_SIZE - 1) // TRAIN_BATCH_SIZE
    total_steps = steps_per_epoch * NUM_EPOCHS
    print(f"✅ Train examples: {len(train_dataset)}")
    print(f"✅ Eval examples:  {len(eval_dataset)}")
    print(f"✅ Steps per epoch: {steps_per_epoch}")
    print(f"✅ Total training steps: {total_steps}")


    data_collator = DataCollatorWithPadding(
        tokenizer=tokenizer,
        pad_to_multiple_of=8,
    )


    accuracy_metric = evaluate.load("accuracy")

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        preds = np.argmax(logits, axis=-1)
        return {"accuracy": accuracy_metric.compute(predictions=preds, references=labels)["accuracy"]}


    print("⚙️ Setting up AdaLoRA...")

    adalora_config = AdaLoraConfig(
        task_type=TaskType.SEQ_CLS,
        r=RANK,
        lora_alpha=32,
        lora_dropout=0.05,
        bias="none",
        target_modules=[
            "q_proj", "k_proj", "v_proj", "o_proj",
            "gate_proj", "up_proj", "down_proj",
        ],

        init_r=12,                              
        target_r=RANK,                          
        tinit=int(0.1 * total_steps),           
        tfinal=int(0.8 * total_steps),          
        deltaT=max(1, int(0.04 * total_steps)), 
        beta1=0.85,
        beta2=0.85,
        orth_reg_weight=0.5,                    
        total_step=total_steps,                
    )

    model = get_peft_model(model, adalora_config)
    model.print_trainable_parameters()

    print("🧾 Building Trainer...")

    training_args = TrainingArguments(
        disable_tqdm=False,
        output_dir=OUTPUT_DIR,
        num_train_epochs=NUM_EPOCHS,
        per_device_train_batch_size=TRAIN_BATCH_SIZE,
        per_device_eval_batch_size=EVAL_BATCH_SIZE,
        logging_steps=500,
        eval_strategy="epoch",
        eval_steps=500,
        learning_rate=LEARNING_RATE,
        bf16=torch.cuda.is_available(), 
        report_to="none",
        save_strategy="no",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics,
    )

    print("🚀 Starting AdaLoRA training on SST-2...")
    trainer.train()

    print("📊 Final evaluation on validation set:")
    metrics = trainer.evaluate()
    print(metrics)


if __name__ == "__main__":
    main()


torch: 2.9.0+cu126
cuda devices: 1
📚 Loading GLUE/SST-2 dataset with retry...
✅ Loaded dataset 'glue'/'sst2' successfully.
🧠 Loading base TinyLlama model...
🔤 Loading tokenizer...
🧹 Preprocessing & tokenizing...


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

✅ Train examples: 10000
✅ Eval examples:  872
✅ Steps per epoch: 625
✅ Total training steps: 1875
⚙️ Setting up AdaLoRA...
trainable params: 9,467,704 || all params: 1,043,984,338 || trainable%: 0.9069
🧾 Building Trainer...
🚀 Starting AdaLoRA training on SST-2...


Epoch,Training Loss,Validation Loss,Accuracy
1,0.448200,0.242332,0.939220
2,0.189600,0.233042,0.943807
3,0.137700,0.259408,0.946101


📊 Final evaluation on validation set:


{'eval_loss': 0.25940799713134766, 'eval_accuracy': 0.9461009174311926, 'eval_runtime': 52.0553, 'eval_samples_per_second': 16.751, 'eval_steps_per_second': 2.094, 'epoch': 3.0}
